In [ ]:
!pip install pandas
!pip install xlrd
!pip install geopandas

In [2]:
import pandas as pd
import geopandas
from geopandas import GeoDataFrame as gpd
import numpy as np
import json

# 1. Carbon Flux to the Atmosphere from Land-Use Changes 1850-2005 Dataset

In [3]:
usecols=['Year','Global','USA','Canada','S+C America','Europe','Trop.Africa','China','S+SE Asia','Pac.Dev.Reg']
landuse = pd.read_excel("../../data/Global_land-use_flux-1850_2005.xls"
                        , sheet_name="net fluxes"
                        , usecols=usecols)
landuse.head()

,Year,Global,USA,Canada,S+C America,Europe,Trop.Africa,China,S+SE Asia,Pac.Dev.Reg
0,1850,500.6,164.0922,5.5476,23.4757,55.0441,-1.3484,101.8392,87.3469,2.0458
1,1851,492.7,165.7256,5.3626,23.1520,55.0156,-1.1192,93.0766,86.9100,2.0419
2,1852,548.5,230.6725,5.3380,22.8618,54.9874,-1.0033,83.8307,86.9385,2.0379
3,1853,546.8,238.5149,5.3138,22.6017,54.9590,-1.0513,74.2140,86.9935,2.0338
4,1854,544.8,246.1846,5.2899,22.3687,54.9304,-0.9855,64.3129,87.0630,2.0297


In [4]:
landuse.columns = ['date','Global','USA','Canada','America','Europe','Africa','China','Asia','Pacific Region']
landuse.head()

,date,Global,USA,Canada,America,Europe,Africa,China,Asia,PacificRegion
0,1850,500.6,164.0922,5.5476,23.4757,55.0441,-1.3484,101.8392,87.3469,2.0458
1,1851,492.7,165.7256,5.3626,23.1520,55.0156,-1.1192,93.0766,86.9100,2.0419
2,1852,548.5,230.6725,5.3380,22.8618,54.9874,-1.0033,83.8307,86.9385,2.0379
3,1853,546.8,238.5149,5.3138,22.6017,54.9590,-1.0513,74.2140,86.9935,2.0338
4,1854,544.8,246.1846,5.2899,22.3687,54.9304,-0.9855,64.3129,87.0630,2.0297


In [5]:
# set year as the index and drop global column
data = landuse.set_index(['date']).drop(columns=['Global'])
data.head()

,USA,Canada,America,Europe,Africa,China,Asia,PacificRegion
date,,,,,,,,
1850,164.0922,5.5476,23.4757,55.0441,-1.3484,101.8392,87.3469,2.0458
1851,165.7256,5.3626,23.1520,55.0156,-1.1192,93.0766,86.9100,2.0419
1852,230.6725,5.3380,22.8618,54.9874,-1.0033,83.8307,86.9385,2.0379
1853,238.5149,5.3138,22.6017,54.9590,-1.0513,74.2140,86.9935,2.0338
1854,246.1846,5.2899,22.3687,54.9304,-0.9855,64.3129,87.0630,2.0297


In [6]:
source_name = "cdiac"
source_url = "https://cdiac.ess-dive.lbl.gov/trends/landuse/houghton/houghton.html"
geo_scale = "country"
geo_id_type = "name"
emission_unit = "Mtc"
gas_name = "CO2"
# land user sector name & mapping
sector_origin_name = "3B-Land Use"
sector_mapped_name = "lucf"
# Create the output file
path = "../../data/json/mapped_data_" + source_name+".json"
file = open(path, "a")
# iterate through list of contient
for country in data.columns:
    subdata = data[country]
    for year, co2 in subdata.items():              
        # Create dictionary that will contain the final json object
        dict_mapped_entry={}  
        ############## Entries associated with geo_component
        dict_geo_component={}
        dict_geo_component["scale"] = geo_scale
        dict_geo_component["name"] = country      
        dict_geo_component["identifier"] = {"id": country
                                            , "type": geo_id_type }
        ############# Entries associated with emission
        dict_emission={}
        dict_emission["gas"] = gas_name
        dict_emission["value"] = str(co2)
        dict_emission["unit"] = {"unit_used" : emission_unit} 
        dict_emission["sector"] = {"sector_origin_name": sector_origin_name
                                    , "sector_mapped_name": sector_mapped_name } 
        ############# Combine everything together
        ############## Entries associated with data_source
        dict_mapped_entry["data_source"] = {"name": source_name
                                            , "link":source_url}
        dict_mapped_entry["geo_component"] = dict_geo_component 
        dict_mapped_entry["date"] = str(year)+"-01-01"
        dict_mapped_entry["emission"] = dict_emission
        # Write the json object to a file and add a line break (every line is a json object)
        json.dump(dict_mapped_entry, file)
        file.write("\n")

file.close()